# Пример работы с allennlp
Здесь мы разберем аналогичную модель, что мы писали на торче.

In [5]:
import torch
from src_allennlp.model import SimpleTagger
from src_allennlp.reader import RuPosReader
from allennlp.data.vocabulary import Vocabulary
from allennlp.common import Params
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer

Из всех импортов тут только два самописных класса - ридер и, собственно, сама модель.

In [2]:
reader = RuPosReader()
dataset = reader.read('data/train.csv')
vocab = Vocabulary.from_instances(dataset, pretrained_files={'tokens':'data/cc.ru.300.vec'}, only_include_pretrained_words=False)
print(vocab)
print(vocab.get_index_to_token_vocabulary('labels'))

48171it [00:03, 12951.79it/s]
100%|████████████████████████████████████████████████████████████████████| 2000000/2000000 [00:15<00:00, 127662.48it/s]


Vocabulary with namespaces:
 	Non Padded Namespaces: {'*labels', '*tags'}
 	Namespace: tokens, Size: 98882 
 	Namespace: labels, Size: 17 

{0: 'NOUN', 1: 'PUNCT', 2: 'VERB', 3: 'ADJ', 4: 'ADP', 5: 'ADV', 6: 'PROPN', 7: 'PRON', 8: 'CONJ', 9: 'PART', 10: 'DET', 11: 'SCONJ', 12: 'NUM', 13: 'AUX', 14: 'X', 15: 'INTJ', 16: 'SYM'}


В ячейке выше мы прочитали датасет и сделали из него словарь

In [3]:
word_emb = 300
hidden_dim = 300

params = Params({"pretrained_file": 'data/cc.ru.300.vec', "embedding_dim": word_emb, "trainable": False})
embedder = BasicTextFieldEmbedder({"tokens": Embedding.from_params(vocab, params)})
encoder = PytorchSeq2SeqWrapper(torch.nn.GRU(embedder.get_output_dim(), hidden_dim, batch_first=True, bidirectional=True, num_layers=2))
model = SimpleTagger(vocab, embedder, encoder)

100%|█████████████████████████████████████████████████████████████████████| 2000000/2000000 [00:26<00:00, 76039.26it/s]


Наша модель состоит из:
- эмбеддера, возвращающего векторные представления для каждого токена
- энкодера, который представляет собой всё ту же GRU, как и раньше, только всякими неудобными вещами занимется `PytorchSeq2SeqWrapper`.
- FeedForward, который уже встроен в модель, но его можно и передать отдельно, предварительно поддержав в самой модели в конструкторе

In [4]:
train_dataset = dataset[:-1000]
dev_dataset = dataset[-1000:]

Распиливаем датасет, слайсы здесь работают нормально

In [6]:
device = torch.device('cuda')
model.to(device)
optimizer = torch.optim.Adam(model.parameters())
iterator = BucketIterator(batch_size=256, sorting_keys=[("tokens", "num_tokens")], biggest_batch_first=True)
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=dev_dataset,
                  patience=5,
                  num_epochs=20,
                  cuda_device=0,
                  validation_metric="+fscore")

Переносим модель на видеокарту, задаём оптимизатор, итератор с индексированием по словарю и сам тренер. У тренера есть куча удобных параметров, например, сколько эпох учить, через сколько эпох остановиться, если таргетная метрика перестаёт расти на валидации (`patience`) и собственно сама таргетная метрика, по которой будет выбираться лучшая эпоха

In [7]:
trainer.train()

accuracy: 0.7411, precision: 0.7411, recall: 0.7411, fscore: 0.7411, loss: 0.8184 ||: 100%|█| 185/185 [00:13<00:00, 13.
accuracy: 0.9293, precision: 0.9293, recall: 0.9293, fscore: 0.9293, loss: 0.2344 ||: 100%|█| 4/4 [00:00<00:00, 23.73it
accuracy: 0.9436, precision: 0.9436, recall: 0.9436, fscore: 0.9436, loss: 0.1905 ||: 100%|█| 185/185 [00:11<00:00, 16.
accuracy: 0.9540, precision: 0.9540, recall: 0.9540, fscore: 0.9540, loss: 0.1566 ||: 100%|█| 4/4 [00:00<00:00, 29.06it
accuracy: 0.9560, precision: 0.9560, recall: 0.9560, fscore: 0.9560, loss: 0.1469 ||: 100%|█| 185/185 [00:11<00:00, 16.
accuracy: 0.9635, precision: 0.9635, recall: 0.9635, fscore: 0.9635, loss: 0.1276 ||: 100%|█| 4/4 [00:00<00:00, 29.93it
accuracy: 0.9614, precision: 0.9614, recall: 0.9614, fscore: 0.9614, loss: 0.1262 ||: 100%|█| 185/185 [00:11<00:00, 16.
accuracy: 0.9665, precision: 0.9665, recall: 0.9665, fscore: 0.9665, loss: 0.1163 ||: 100%|█| 4/4 [00:00<00:00, 29.93it
accuracy: 0.9652, precision: 0.9652, rec

{'best_epoch': 17,
 'peak_cpu_memory_MB': 0,
 'training_duration': '0:03:52.182084',
 'training_start_epoch': 0,
 'training_epochs': 19,
 'epoch': 19,
 'training_accuracy': 0.9918004360780812,
 'training_precision': 0.9918004274368286,
 'training_recall': 0.9918004274368286,
 'training_fscore': 0.9918004274368286,
 'training_loss': 0.019412214443641336,
 'training_cpu_memory_MB': 0.0,
 'validation_accuracy': 0.9776970954356846,
 'validation_precision': 0.9776970744132996,
 'validation_recall': 0.9776970744132996,
 'validation_fscore': 0.9776970744132996,
 'validation_loss': 0.09564318507909775,
 'best_validation_accuracy': 0.9783886583679114,
 'best_validation_precision': 0.9783886671066284,
 'best_validation_recall': 0.9783886671066284,
 'best_validation_fscore': 0.9783886671066284,
 'best_validation_loss': 0.08592475764453411}

После обучения модели остаётся только предстказать метки классов и можно заворачивать

In [10]:
model.eval()
results = []
with torch.no_grad():    
    labels =  model.forward_on_instance(dev_dataset[1])['labels']

Сам по себе инстанс это что-то похожее на словарь, поэтому чтобы добраться до токенов, нужно немного пройти по словарю

In [14]:
for token, label in zip(dev_dataset[1]['tokens'].tokens,labels):
    print(token, label)

Сегодня ADV
Великий ADJ
Октябрь NOUN
прибавил VERB
к ADP
своей DET
биографии NOUN
еще ADV
один NUM
год NOUN
. PUNCT


Собственно вот и все. Allennlp прекрасен и очень удобен